# 14 MONTE CARLO SIMULATION

Stanislaw Ulam and Nicholas Metropolis coined the term <b>Monte Carlo simulation</b> in 1949 in homage to the games of chance played in the casino in the Principality of Monaco.

## 14.1 Pascal’s Problem

Pascal’s interest in the field that came to be known as probability theory began when a friend asked him <b>whether or not it would be profitable to bet that within twenty-four rolls of a pair of dice he would roll a double six</b>

In the long run it <b>would not be profitable to bet</b> on rolling a double six within twenty-four rolls

In [15]:
import random
#Page 194, figure 14.1
def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    """Assumes numTrials an int > 0
       Prints an estimate of the probability of winning"""
    numWins = 0.0
    for i in range(numTrials):
        for j in range(24):
            d1 = rollDie()
            d2 = rollDie()
            if d1 == 6 and d2 == 6:
                numWins += 1
                break
    print('Probability of winning =', numWins/numTrials)

checkPascal(1000)

Probability of winning = 0.486


In [16]:
1 - (35.0/36.0)**24

0.4914038761309034

## 14.2 Pass or Don’t Pass?

In the game craps, the shooter (the person who rolls the dice) chooses between making a “pass line” or a “don’t pass line” bet.

In [23]:
import pylab, random
#Page 196, Figure 14.2
class CrapsGame(object):
    def __init__(self):
        self.passWins, self.passLosses = (0,0)
        self.dpWins, self.dpLosses, self.dpPushes = (0,0,0)

    def playHand(self):
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
            self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            point = throw
            while True:
                throw = rollDie() + rollDie()
                if throw == point:
                    self.passWins += 1
                    self.dpLosses += 1
                    break
                elif throw == 7:
                    self.passLosses += 1
                    self.dpWins += 1
                    break

    def passResults(self):
        return (self.passWins, self.passLosses)

    def dpResults(self):
        return (self.dpWins, self.dpLosses, self.dpPushes)



Figure 14.3 contains a function that uses class `CrapsGame`. 

Its structure is typical of many simulation programs

In [28]:
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = float(sum(X))/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return (tot/len(X))**0.5 #Square root of mean difference

#Page 197, Figure 14.3
def crapsSim(handsPerGame, numGames):
    """Assumes handsPerGame and numGames are ints > 0
       Play numGames games of handsPerGame hands, and print results"""
    games = []

    #Play numGames games
    for t in xrange(numGames):
        c = CrapsGame()
        for i in xrange(handsPerGame):
            c.playHand()
        games.append(c)
        
    #Produce statistics for each game
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
        
    #Produce and print summary statistics
    meanROI = str(round((100.0*sum(pROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)
    meanROI = str(round((100.0*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(dpROIPerGame), 4)) + '%'
    print('Don\'t pass:','Mean ROI =', meanROI, 'Std Dev =', sigma)

Let’s run our craps simulation and see what happens

In [29]:
crapsSim(20, 10)

Pass: Mean ROI = 1.0% Std. Dev. = 21.1896%
Don't pass: Mean ROI = -2.0% Std Dev = 20.7605%


## 14.3 Using Table Lookup to Improve Performance

That raises the question of whether there is a simple way to speed up the simulation.

The idea of replacing computation by <b>table lookup</b> has broad applicability and is frequently used when speed is an issue. Table lookup is an example of the
general idea of <b>trading time for space</b>.

Using table lookup to improve performance

In [ ]:
#Page 196, Figure 14.2
class FastCrapsGame(CrapsGame):
    
#Page 200, Figure 14.4
    def playHand(self):
        #An alternative, faster, implementation of playHand
        pointsDict = {4:1/3.0, 5:2/5.0, 6:5/11.0, 8:5/11.0,
                  9:2/5.0, 10:1/3.0}
        throw = rollDie() + rollDie()
        if throw == 7 or throw == 11:
            self.passWins += 1
            self.dpLosses += 1
        elif throw == 2 or throw == 3 or throw == 12:
                self.passLosses += 1
            if throw == 12:
                self.dpPushes += 1
            else:
                self.dpWins += 1
        else:
            if random.random() <= pointsDict[throw]: # point before 7
                self.passWins += 1
                self.dpLosses += 1
            else:                                    # 7 before point
                self.passLosses += 1
                self.dpWins += 1


In [ ]:
def stdDev(X):
    """Assumes that X is a list of numbers.
       Returns the standard deviation of X"""
    mean = float(sum(X))/len(X)
    tot = 0.0
    for x in X:
        tot += (x - mean)**2
    return (tot/len(X))**0.5 #Square root of mean difference

#Page 197, Figure 14.3
def crapsSim(handsPerGame, numGames):
    """Assumes handsPerGame and numGames are ints > 0
       Play numGames games of handsPerGame hands, and print results"""
    games = []

    #Play numGames games
    for t in xrange(numGames):
        c = CrapsGame()
        for i in xrange(handsPerGame):
            c.playHand()
        games.append(c)
        
    #Produce statistics for each game
    pROIPerGame, dpROIPerGame = [], []
    for g in games:
        wins, losses = g.passResults()
        pROIPerGame.append((wins - losses)/float(handsPerGame))
        wins, losses, pushes = g.dpResults()
        dpROIPerGame.append((wins - losses)/float(handsPerGame))
        
    #Produce and print summary statistics
    meanROI = str(round((100.0*sum(pROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(pROIPerGame), 4)) + '%'
    print('Pass:', 'Mean ROI =', meanROI, 'Std. Dev. =', sigma)
    meanROI = str(round((100.0*sum(dpROIPerGame)/numGames), 4)) + '%'
    sigma = str(round(100.0*stdDev(dpROIPerGame), 4)) + '%'
    print('Don\'t pass:','Mean ROI =', meanROI, 'Std Dev =', sigma)

## 14.4 Finding π

Monte Carlo simulation (and randomized algorithms in general) can be used to solve problems that are not inherently stochastic, i.e., for which there is no
uncertainty about outcomes.

In [33]:
# Page 203, Figure 14.5
def throwNeedles(numNeedles):
    inCircle = 0
    for Needles in range(1, numNeedles + 1):
        x = random.random()
        y = random.random()
        if (x*x + y*y)**0.5 <= 1.0:
            inCircle += 1
    #Counting needles in one quadrant only, so multiply by 4
    return 4*(inCircle/float(numNeedles))

def getEst(numNeedles, numTrials):
    estimates = []
    for t in range(numTrials):
        piGuess = throwNeedles(numNeedles)
        estimates.append(piGuess)
    sDev = stdDev(estimates)
    curEst = sum(estimates)/len(estimates)
    print('Est. = ' + str(round(curEst, 5)) +
          ', Std. dev. = ' + str(round(sDev, 5))
          + ', Needles = ' + str(numNeedles))
    return (curEst, sDev)

def estPi(precision, numTrials):
    numNeedles = 1000
    sDev = precision
    while sDev >= precision/2.0:
        curEst, sDev = getEst(numNeedles, numTrials)
        numNeedles *= 2
    return curEst


In [35]:
estPi(0.01, 100)

Est. = 3.1368, Std. dev. = 0.05668, Needles = 1000
Est. = 3.14526, Std. dev. = 0.03752, Needles = 2000
Est. = 3.14589, Std. dev. = 0.02556, Needles = 4000
Est. = 3.14122, Std. dev. = 0.01656, Needles = 8000
Est. = 3.13899, Std. dev. = 0.01413, Needles = 16000
Est. = 3.14193, Std. dev. = 0.00869, Needles = 32000
Est. = 3.14152, Std. dev. = 0.00636, Needles = 64000
Est. = 3.14169, Std. dev. = 0.00467, Needles = 128000


3.1416853125

## 14.5 Some Closing Remarks About Simulation Models

For most of the history of science, theorists used mathematical techniques to construct purely analytical models that could be used to predict the behavior of a system from a set of parameters and initial conditions.

As the 20th century progressed, the limitations of this approach became increasingly clear. Reasons for this include:
```
1 An increased interest in <b>the social sciences</b>

2 As the systems to be modeled grew <b>increasingly complex</b>, it seemed easier to successively refine a series of simulation models than to construct accurate analytic models

3 It is often easier to extract useful intermediate results from a simulation than from an analytical model

4 The <b>availability of computers</b> made it feasible to run large-scale simulations
```
Simulation attempts to build an experimental device, called <b>a model</b>, that will provide useful information about the possible behaviors of the system being modeled.

Simulation models are <b>descriptive</b>, <b>not prescriptive</b>. They tell how a system works under given conditions; not how to arrange the conditions to make the system work best.

Simulation models can be classified along <b>three dimensions</b>:

• Deterministic versus stochastic,

• Static versus dynamic, and

• Discrete versus continuous.